<a href="https://colab.research.google.com/github/profliuhao/CSIT555/blob/main/CSIT_555_Module_8_Indexing_and_Hashing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 8: Indexing and Hashing

### [name]
### [student ID]

In [1]:
# Let's start with load the extension sql
%load_ext sql
%config SqlMagic.style='_DEPRECATED_DEFAULT'
# and then connect to a database **lab3_SQL.db** using:
%sql sqlite:///lab8_SQL.db

In [2]:
%%sql
-- Enable foreign key support
PRAGMA foreign_keys = ON;

drop table if exists customers;
drop table if exists orders;

-- Create a customers table
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL,
    phone TEXT,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);

-- Create an orders table with all necessary fields
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    order_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    status TEXT CHECK(status IN ('pending', 'processing', 'shipped', 'delivered')),
    amount DECIMAL(10,2),
    shipping_address TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);


 * sqlite:///lab8_SQL.db
Done.
Done.
Done.
Done.
Done.


[]

In [3]:
%%sql


-- Generate sample data for customers
INSERT INTO customers (name, email, phone)
WITH RECURSIVE
  chars(x) AS (
    SELECT 1 UNION ALL SELECT x+1 FROM chars WHERE x < 10000
  )
SELECT
  'Customer ' || x,
  'customer' || x || '@' ||
    CASE (ABS(RANDOM()) % 4)
      WHEN 0 THEN 'gmail.com'
      WHEN 1 THEN 'yahoo.com'
      WHEN 2 THEN 'hotmail.com'
      ELSE 'example.com'
    END,
  '+1-' || (ABS(RANDOM()) % 800 + 200) || '-' ||
    (ABS(RANDOM()) % 900 + 100) || '-' ||
    (ABS(RANDOM()) % 9000 + 1000)
FROM chars;

-- Generate sample orders (multiple orders per customer)
INSERT INTO orders (customer_id, order_date, status, amount, shipping_address)
WITH RECURSIVE
  entries(x) AS (
    SELECT 1 UNION ALL SELECT x+1 FROM entries WHERE x < 100000
  )
SELECT
  (ABS(RANDOM()) % 10000) + 1,
  datetime('now', '-' || (ABS(RANDOM()) % 365) || ' days'),
  CASE (ABS(RANDOM()) % 4)
    WHEN 0 THEN 'pending'
    WHEN 1 THEN 'processing'
    WHEN 2 THEN 'shipped'
    ELSE 'delivered'
  END,
  ROUND((ABS(RANDOM()) % 900) + (ABS(RANDOM()) % 100) / 100.0 + 10.0, 2),
  (ABS(RANDOM()) % 9900 + 100) || ' ' ||
  CASE (ABS(RANDOM()) % 5)
    WHEN 0 THEN 'Main St'
    WHEN 1 THEN 'Oak Ave'
    WHEN 2 THEN 'Park Rd'
    WHEN 3 THEN 'Lake Dr'
    ELSE 'Hill Blvd'
  END || ', ' ||
  CASE (ABS(RANDOM()) % 5)
    WHEN 0 THEN 'New York, NY'
    WHEN 1 THEN 'Los Angeles, CA'
    WHEN 2 THEN 'Chicago, IL'
    WHEN 3 THEN 'Houston, TX'
    ELSE 'Phoenix, AZ'
  END || ' ' ||
  (ABS(RANDOM()) % 89999 + 10000)
FROM entries;



 * sqlite:///lab8_SQL.db
10000 rows affected.
100000 rows affected.


[]

In [4]:
%%sql
select * from customers limit 20;

 * sqlite:///lab8_SQL.db
Done.


customer_id,name,email,phone,created_at
1,Customer 1,customer1@gmail.com,+1-607-443-4969,2025-03-01 20:00:04
2,Customer 2,customer2@example.com,+1-712-869-9055,2025-03-01 20:00:04
3,Customer 3,customer3@hotmail.com,+1-939-427-4812,2025-03-01 20:00:04
4,Customer 4,customer4@gmail.com,+1-788-626-7910,2025-03-01 20:00:04
5,Customer 5,customer5@yahoo.com,+1-211-350-1405,2025-03-01 20:00:04
6,Customer 6,customer6@gmail.com,+1-580-418-2576,2025-03-01 20:00:04
7,Customer 7,customer7@yahoo.com,+1-884-130-9434,2025-03-01 20:00:04
8,Customer 8,customer8@gmail.com,+1-780-911-6333,2025-03-01 20:00:04
9,Customer 9,customer9@example.com,+1-223-414-9833,2025-03-01 20:00:04
10,Customer 10,customer10@yahoo.com,+1-375-451-9002,2025-03-01 20:00:04


In [5]:
%%sql
select * from orders limit 50;

 * sqlite:///lab8_SQL.db
Done.


order_id,customer_id,order_date,status,amount,shipping_address
1,3340,2024-04-10 20:00:04,pending,424.12,"7659 Oak Ave, Houston, TX 70478"
2,8929,2024-04-01 20:00:04,shipped,153.91,"7258 Oak Ave, Houston, TX 91292"
3,4995,2024-11-13 20:00:04,delivered,468.5,"2209 Oak Ave, New York, NY 35615"
4,8085,2024-08-31 20:00:04,pending,856.43,"399 Park Rd, Phoenix, AZ 79370"
5,1267,2024-05-17 20:00:04,pending,439.4,"5590 Oak Ave, Phoenix, AZ 19085"
6,441,2025-02-15 20:00:04,shipped,511.47,"9073 Hill Blvd, New York, NY 61741"
7,8844,2024-09-18 20:00:04,shipped,377.02,"7761 Park Rd, Chicago, IL 13704"
8,3408,2025-02-12 20:00:04,processing,91.45,"7266 Lake Dr, New York, NY 41009"
9,2386,2024-06-21 20:00:04,delivered,594.5,"6500 Oak Ave, New York, NY 88885"
10,6977,2024-10-11 20:00:04,delivered,159.14,"152 Hill Blvd, Chicago, IL 45165"


In [6]:
%%sql
select count(1) from orders;

 * sqlite:///lab8_SQL.db
Done.


count(1)
100000


In [7]:
%%sql
select count(*) from orders
where amount > 909;

 * sqlite:///lab8_SQL.db
Done.


count(*)
105


In [8]:
%%time
%%sql

-- Create indexes and analyze their impact

-- Analyze query performance before and after index
EXPLAIN QUERY PLAN
SELECT * FROM customers WHERE email = 'customer900@gmail.com';

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 892 µs, sys: 1.82 ms, total: 2.71 ms
Wall time: 2.41 ms


id,parent,notused,detail
2,0,0,SCAN customers


In [9]:
%%time
%%sql

-- Create indexes and analyze their impact

-- Analyze query performance before and after index
SELECT * FROM customers WHERE email = 'customer900@gmail.com';

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 3.11 ms, sys: 856 µs, total: 3.96 ms
Wall time: 3.4 ms


customer_id,name,email,phone,created_at


In [10]:
%%time
%%sql
-- Analyze compound index usage
EXPLAIN QUERY PLAN
SELECT * FROM orders
WHERE customer_id = 1000
AND order_date >= datetime('now', '-30 days');

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 2.96 ms, sys: 152 µs, total: 3.11 ms
Wall time: 2.96 ms


id,parent,notused,detail
2,0,0,SCAN orders


In [11]:
%%time
%%sql
-- Analyze compound index usage
SELECT * FROM orders
WHERE customer_id = 1000
AND order_date >= datetime('now', '-30 days');

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 10.4 ms, sys: 4.85 ms, total: 15.3 ms
Wall time: 14.9 ms


order_id,customer_id,order_date,status,amount,shipping_address
29772,1000,2025-02-05 20:00:04,pending,408.39,"5496 Park Rd, Chicago, IL 16289"


In [12]:
%%sql
-- Analyze range query performance
EXPLAIN QUERY PLAN
SELECT * FROM orders
WHERE amount BETWEEN 490 AND 500
ORDER BY amount DESC;

 * sqlite:///lab8_SQL.db
Done.


id,parent,notused,detail
3,0,0,SCAN orders
17,0,0,USE TEMP B-TREE FOR ORDER BY


In [13]:
%%time
%%sql
-- Analyze range query performance
SELECT * FROM orders
WHERE amount BETWEEN 490 AND 500
ORDER BY amount DESC
limit 100;

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 18.2 ms, sys: 5.98 ms, total: 24.2 ms
Wall time: 33 ms


order_id,customer_id,order_date,status,amount,shipping_address
19655,6603,2024-11-28 20:00:04,shipped,500,"4178 Oak Ave, Los Angeles, CA 13447"
46862,4792,2024-05-09 20:00:04,delivered,500,"8879 Park Rd, Phoenix, AZ 81633"
67472,6331,2024-11-01 20:00:04,pending,500,"7341 Oak Ave, New York, NY 60874"
40266,4444,2024-06-15 20:00:04,delivered,499.99,"5987 Lake Dr, Phoenix, AZ 19469"
9893,8741,2024-06-26 20:00:04,pending,499.97,"4834 Oak Ave, Chicago, IL 72005"
32977,8407,2024-10-22 20:00:04,pending,499.96,"9666 Lake Dr, Chicago, IL 90977"
53771,2288,2024-09-08 20:00:04,shipped,499.96,"9308 Park Rd, New York, NY 47289"
9226,782,2024-03-02 20:00:04,processing,499.95,"9703 Oak Ave, Phoenix, AZ 69691"
30816,2326,2025-01-04 20:00:04,pending,499.94,"3008 Main St, Houston, TX 75093"
68873,1434,2025-01-24 20:00:04,shipped,499.91,"9109 Lake Dr, Phoenix, AZ 99881"


In [14]:
%%sql
-- Analyze phone lookup performance
EXPLAIN QUERY PLAN
SELECT * FROM customers
WHERE phone LIKE '+1-555%';

 * sqlite:///lab8_SQL.db
Done.


id,parent,notused,detail
2,0,0,SCAN customers


In [15]:
%%time
%%sql
-- Analyze phone lookup performance
SELECT * FROM customers
WHERE phone LIKE '+1-555%';

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 4.73 ms, sys: 0 ns, total: 4.73 ms
Wall time: 4.68 ms


customer_id,name,email,phone,created_at
377,Customer 377,customer377@hotmail.com,+1-555-747-3556,2025-03-01 20:00:04
1370,Customer 1370,customer1370@yahoo.com,+1-555-398-6500,2025-03-01 20:00:04
1501,Customer 1501,customer1501@yahoo.com,+1-555-129-6669,2025-03-01 20:00:04
2099,Customer 2099,customer2099@example.com,+1-555-854-1381,2025-03-01 20:00:04
3421,Customer 3421,customer3421@example.com,+1-555-989-7059,2025-03-01 20:00:04
3440,Customer 3440,customer3440@yahoo.com,+1-555-695-5038,2025-03-01 20:00:04
3461,Customer 3461,customer3461@gmail.com,+1-555-861-5563,2025-03-01 20:00:04
4661,Customer 4661,customer4661@example.com,+1-555-673-6731,2025-03-01 20:00:04
7555,Customer 7555,customer7555@hotmail.com,+1-555-927-8295,2025-03-01 20:00:04
7700,Customer 7700,customer7700@example.com,+1-555-724-3691,2025-03-01 20:00:04


In [16]:
%%sql
-- Analyze partial index usage
EXPLAIN QUERY PLAN
SELECT * FROM orders
WHERE amount > 909.5;

 * sqlite:///lab8_SQL.db
Done.


id,parent,notused,detail
2,0,0,SCAN orders


In [17]:
%%time
%%sql
-- Analyze partial index usage
SELECT * FROM orders
WHERE amount > 909.5
limit 100;

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 10.2 ms, sys: 4.82 ms, total: 15 ms
Wall time: 16.2 ms


order_id,customer_id,order_date,status,amount,shipping_address
638,1993,2024-04-19 20:00:04,processing,909.85,"7452 Main St, Houston, TX 78894"
2658,8289,2024-04-04 20:00:04,delivered,909.73,"3133 Park Rd, Phoenix, AZ 50483"
2841,617,2024-05-06 20:00:04,shipped,909.77,"335 Main St, Phoenix, AZ 10100"
6528,8446,2024-03-06 20:00:04,delivered,909.81,"7201 Hill Blvd, Chicago, IL 39790"
6876,3576,2024-05-01 20:00:04,delivered,909.55,"4383 Hill Blvd, Phoenix, AZ 85224"
9256,246,2024-10-22 20:00:04,pending,909.79,"3624 Lake Dr, New York, NY 24897"
9284,3220,2024-04-25 20:00:04,pending,909.73,"644 Oak Ave, Phoenix, AZ 11395"
9610,7821,2024-08-08 20:00:04,shipped,909.6,"623 Main St, New York, NY 95280"
10861,7318,2025-02-27 20:00:04,delivered,909.65,"8393 Park Rd, Los Angeles, CA 99795"
10997,4824,2024-12-21 20:00:04,shipped,909.88,"2509 Main St, Houston, TX 97759"


In [18]:
%%sql
-- 1. Basic index on customer email (commonly used for lookups)
CREATE INDEX idx_customers_email ON customers(email);

-- Analyze query performance before and after index
EXPLAIN QUERY PLAN
SELECT * FROM customers WHERE email = 'customer900@gmail.com';

 * sqlite:///lab8_SQL.db
Done.
Done.


id,parent,notused,detail
3,0,0,SEARCH customers USING INDEX idx_customers_email (email=?)


In [19]:
%%time
%%sql

-- Create indexes and analyze their impact

-- Analyze query performance before and after index
SELECT * FROM customers WHERE email = 'customer900@gmail.com';

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 2.32 ms, sys: 0 ns, total: 2.32 ms
Wall time: 1.95 ms


customer_id,name,email,phone,created_at


In [20]:
%%sql
-- 2. Create a compound index for order queries
CREATE INDEX idx_orders_customer_date ON orders(customer_id, order_date);

-- Analyze compound index usage
EXPLAIN QUERY PLAN
SELECT * FROM orders
WHERE customer_id = 1000
AND order_date >= datetime('now', '-30 days');

 * sqlite:///lab8_SQL.db
Done.
Done.


id,parent,notused,detail
3,0,0,SEARCH orders USING INDEX idx_orders_customer_date (customer_id=? AND order_date>?)


In [21]:
%%time
%%sql
-- Analyze compound index usage
SELECT * FROM orders
WHERE customer_id = 1000
AND order_date >= datetime('now', '-30 days');

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 1.75 ms, sys: 117 µs, total: 1.87 ms
Wall time: 1.6 ms


order_id,customer_id,order_date,status,amount,shipping_address
29772,1000,2025-02-05 20:00:04,pending,408.39,"5496 Park Rd, Chicago, IL 16289"


In [26]:
%%sql
-- 3. Create an index for amount range queries
CREATE INDEX idx_orders_amount ON orders(amount);

-- Analyze range query performance
EXPLAIN QUERY PLAN
SELECT * FROM orders
WHERE amount BETWEEN 490 AND 500
ORDER BY amount DESC;

 * sqlite:///lab8_SQL.db
Done.
Done.


id,parent,notused,detail
4,0,0,SEARCH orders USING INDEX idx_orders_amount (amount>? AND amount<?)


In [27]:
%%time
%%sql
-- Analyze range query performance
SELECT * FROM orders
WHERE amount BETWEEN 490 AND 500
ORDER BY amount DESC
limit 100;

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 3.74 ms, sys: 115 µs, total: 3.86 ms
Wall time: 3.32 ms


order_id,customer_id,order_date,status,amount,shipping_address
67472,6331,2024-11-01 20:00:04,pending,500,"7341 Oak Ave, New York, NY 60874"
46862,4792,2024-05-09 20:00:04,delivered,500,"8879 Park Rd, Phoenix, AZ 81633"
19655,6603,2024-11-28 20:00:04,shipped,500,"4178 Oak Ave, Los Angeles, CA 13447"
40266,4444,2024-06-15 20:00:04,delivered,499.99,"5987 Lake Dr, Phoenix, AZ 19469"
9893,8741,2024-06-26 20:00:04,pending,499.97,"4834 Oak Ave, Chicago, IL 72005"
53771,2288,2024-09-08 20:00:04,shipped,499.96,"9308 Park Rd, New York, NY 47289"
32977,8407,2024-10-22 20:00:04,pending,499.96,"9666 Lake Dr, Chicago, IL 90977"
9226,782,2024-03-02 20:00:04,processing,499.95,"9703 Oak Ave, Phoenix, AZ 69691"
30816,2326,2025-01-04 20:00:04,pending,499.94,"3008 Main St, Houston, TX 75093"
68873,1434,2025-01-24 20:00:04,shipped,499.91,"9109 Lake Dr, Phoenix, AZ 99881"


In [28]:
%%sql
-- 4. Create an index on phone numbers with a WHERE clause
CREATE INDEX idx_customers_phone ON customers(phone)
WHERE phone IS NOT NULL;

-- Analyze phone lookup performance
EXPLAIN QUERY PLAN
SELECT * FROM customers
WHERE phone LIKE '+1-555%';


 * sqlite:///lab8_SQL.db
Done.
Done.


id,parent,notused,detail
3,0,0,SCAN customers USING INDEX idx_customers_phone


In [29]:
%%time
%%sql
-- Analyze phone lookup performance
SELECT * FROM customers
WHERE phone LIKE '+1-555%';

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 4.3 ms, sys: 0 ns, total: 4.3 ms
Wall time: 3.56 ms


customer_id,name,email,phone,created_at
1501,Customer 1501,customer1501@yahoo.com,+1-555-129-6669,2025-03-01 20:00:04
9085,Customer 9085,customer9085@hotmail.com,+1-555-197-2194,2025-03-01 20:00:04
8265,Customer 8265,customer8265@example.com,+1-555-228-8709,2025-03-01 20:00:04
1370,Customer 1370,customer1370@yahoo.com,+1-555-398-6500,2025-03-01 20:00:04
9356,Customer 9356,customer9356@yahoo.com,+1-555-449-1429,2025-03-01 20:00:04
9196,Customer 9196,customer9196@example.com,+1-555-626-7830,2025-03-01 20:00:04
4661,Customer 4661,customer4661@example.com,+1-555-673-6731,2025-03-01 20:00:04
3440,Customer 3440,customer3440@yahoo.com,+1-555-695-5038,2025-03-01 20:00:04
7700,Customer 7700,customer7700@example.com,+1-555-724-3691,2025-03-01 20:00:04
377,Customer 377,customer377@hotmail.com,+1-555-747-3556,2025-03-01 20:00:04


In [30]:
%%sql
-- Analyze phone lookup performance
EXPLAIN QUERY PLAN
SELECT * FROM customers
WHERE phone = '+1-555-927-8295';

 * sqlite:///lab8_SQL.db
Done.


id,parent,notused,detail
3,0,0,SEARCH customers USING INDEX idx_customers_phone (phone=?)


In [31]:
%%time
%%sql
-- Analyze phone lookup performance
SELECT * FROM customers
WHERE phone = '+1-555-927-8295';

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 2.98 ms, sys: 0 ns, total: 2.98 ms
Wall time: 2.73 ms


customer_id,name,email,phone,created_at
7555,Customer 7555,customer7555@hotmail.com,+1-555-927-8295,2025-03-01 20:00:04


In [32]:
%%sql

PRAGMA index_list('orders');

 * sqlite:///lab8_SQL.db
Done.


seq,name,unique,origin,partial
0,idx_orders_amount,0,c,0
1,idx_high_value_orders,0,c,1
2,idx_orders_customer_date,0,c,0


In [22]:
%%sql

-- drop index idx_orders_amount;
-- 5. Create a partial index for high-value orders
CREATE INDEX idx_high_value_orders ON orders(order_id, customer_id)
WHERE amount > 909;

-- Analyze partial index usage
EXPLAIN QUERY PLAN
SELECT * FROM orders
WHERE amount > 909.5 ;

 * sqlite:///lab8_SQL.db
Done.
Done.


id,parent,notused,detail
2,0,0,SCAN orders


In [24]:
%%time
%%sql
-- Analyze partial index usage
EXPLAIN QUERY PLAN
SELECT * FROM orders
WHERE amount > 909 and amount <909.1
limit 100;

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 1.71 ms, sys: 0 ns, total: 1.71 ms
Wall time: 1.57 ms


id,parent,notused,detail
4,0,0,SCAN orders USING INDEX idx_high_value_orders


In [25]:
%%time
%%sql
-- Analyze partial index usage
SELECT * FROM orders
WHERE amount > 909 and amount <909.1
limit 100;

 * sqlite:///lab8_SQL.db
Done.
CPU times: user 3.21 ms, sys: 37 µs, total: 3.25 ms
Wall time: 2.77 ms


order_id,customer_id,order_date,status,amount,shipping_address
4539,5177,2024-06-08 20:00:04,shipped,909.01,"5970 Park Rd, Phoenix, AZ 98089"
10673,7958,2024-07-29 20:00:04,delivered,909.04,"3348 Main St, New York, NY 78603"
10865,3172,2024-07-21 20:00:04,processing,909.04,"7119 Oak Ave, Los Angeles, CA 19485"
18017,863,2024-12-06 20:00:04,delivered,909.04,"856 Lake Dr, New York, NY 93900"
19534,1606,2024-11-15 20:00:04,processing,909.06,"8869 Hill Blvd, Los Angeles, CA 43620"
38832,1012,2024-09-25 20:00:04,pending,909.05,"452 Oak Ave, Los Angeles, CA 88740"
40312,4004,2025-01-19 20:00:04,processing,909.01,"1578 Hill Blvd, Los Angeles, CA 94336"
42076,3432,2024-06-12 20:00:04,processing,909.08,"9542 Hill Blvd, Houston, TX 33339"
53423,8701,2024-05-01 20:00:04,pending,909.06,"1083 Oak Ave, Chicago, IL 27783"
54929,4238,2025-01-09 20:00:04,delivered,909.01,"9188 Hill Blvd, Chicago, IL 37629"


In [33]:
%%sql
-- Analyze partial index usage
EXPLAIN QUERY PLAN
SELECT order_id, customer_id FROM orders
WHERE amount > 800 AND customer_id > 100;

 * sqlite:///lab8_SQL.db
Done.


id,parent,notused,detail
3,0,0,SEARCH orders USING INDEX idx_orders_amount (amount>?)


In [34]:
%%sql
-- Get index information
SELECT * FROM sqlite_master
WHERE type = 'index';


 * sqlite:///lab8_SQL.db
Done.


type,name,tbl_name,rootpage,sql
index,idx_customers_email,customers,2246,CREATE INDEX idx_customers_email ON customers(email)
index,idx_orders_customer_date,orders,2323,"CREATE INDEX idx_orders_customer_date ON orders(customer_id, order_date)"
index,idx_high_value_orders,orders,3078,"CREATE INDEX idx_high_value_orders ON orders(order_id, customer_id)WHERE amount > 909"
index,idx_orders_amount,orders,3079,CREATE INDEX idx_orders_amount ON orders(amount)
index,idx_customers_phone,customers,3489,CREATE INDEX idx_customers_phone ON customers(phone)WHERE phone IS NOT NULL


In [35]:
%%sql

-- Example of a complex query using multiple indexes
EXPLAIN QUERY PLAN
SELECT
    c.name,
    c.email,
    o.order_date,
    o.amount,
    o.shipping_address
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.status = 'shipped'
AND o.order_date >= datetime('now', '-7 days')
ORDER BY o.amount DESC;



 * sqlite:///lab8_SQL.db
Done.


id,parent,notused,detail
5,0,0,SCAN o USING INDEX idx_orders_amount
16,0,0,SEARCH c USING INTEGER PRIMARY KEY (rowid=?)
